# Data

- $C=\{0,1,2,3,4,5,6,7\}$ $\leftarrow$ set of cities to visit.

- $n = 8$ $\leftarrow$ number of cities.

In [1]:
cities = [c for c in range(8)]
cities

[0, 1, 2, 3, 4, 5, 6, 7]

- $D=(d_{ij})$, $i,j \in C$ $\leftarrow$ matrix of distances between cities.

In [2]:
distances = [
    [0, 89, 87, 38, 33, 71, 59, 54],
    [89, 0, 32, 59, 65, 39, 45, 61],
    [87, 32, 0, 50, 75, 17, 64, 79],
    [38, 59, 50, 0, 40, 33, 50, 56],
    [33, 65, 75, 40, 0, 62, 26, 21],
    [71, 39, 17, 33, 62, 0, 57, 70],
    [59, 45, 64, 50, 26, 57, 0, 16],
    [54, 61, 79, 56, 21, 70, 16, 0]
]

# Decision

- $T=\{t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9=t_1\}$, $t_i \in C$ $\leftarrow$ ordered set of visited cities.

- $T = \{\}$ $\leftarrow$ initialize tour as an empty set.

In [3]:
tour = []

# Objective

The objective is to get a tour $T$ that visits all cities in $C$, ending with the first city, such that the total distance of the tour is as minimal as possible.

$$
\begin{aligned}
& \underset{t \in T}{\text{min}}
& & \sum_{i=1}^{n} d_{t_i t_{i+1}}
\end{aligned}
$$

# Procedure

## Initial step

To start the tour, the two nearest cities will be chosen first; that is, we will find the smallest edge in the matrix of distances, excluding the positions of the same cities:

In [4]:
minimums = (min(enumerate(row), key=lambda l: l[1] if l[1] > 0 else float('inf')) for _, row in enumerate(distances))
smallest = min(enumerate(minimums), key=lambda m: m[1][1])
smallest[1][1]

16

The smallest edge is $16$, but what we need are the coordinates (the cities) to which that edge belongs:

In [5]:
c1 = smallest[0]
c2 = smallest[1][0]
c1, c2

(6, 7)

Now we can append cities $6$ and $7$ to our tour $T$:

$T=T\oplus\{6, 7\}$

In [6]:
tour.extend([6, 7])
tour

[6, 7]

In the TSP, the tour ends with the first visited city, so our tour $T$ should look like $\{6, 7, \ldots, 6\}$:

In [7]:
tour.append(6)
tour

[6, 7, 6]

The first selected cities $6$ and $7$ need to be removed from the set of cities $C$. In order to avoid modifying $C$, we will use a new set to store the unvisited cities:

$\bar{C}=C\setminus\{6,7\}$

In [8]:
cities_bar = cities
cities_bar.remove(6)
cities_bar.remove(7)
cities_bar

[0, 1, 2, 3, 4, 5]

## Nearest Insertion Heuristic

In order to solve the TSP, we will to apply the **nearest insertion heuristic**.

This technique consists of selecting the closest city to the current tour, which is actually a subtour, because we haven't visited all cities yet.

### Distance function

The closest city is the one whose sum of distances to every visited city in the subtour is the smallest.

$$
\newcommand{\dist}{\mathop{\mathrm{dist}}}
\dist(c \in \bar{C}) = \sum_{t \in T} d_{ct}
$$

In [9]:
def dist(candidate_city):
    '''Calculates the distance between candidate_city and the current subtour.'''
    return sum(distances[candidate_city][t] for t in tour)

### Cost function

Having the next city to visit already selected, we proceed to choose an edge of the subtour in which the selected city will be inserted. This is determined by calculating the cost of removing the existing edge and creating two new ones, in order to connect the selected city $c_s$, and choose the smallest cost.

$$
\newcommand{\cost}{\mathop{\mathrm{cost}}}
\cost(c_s, E) = d_{c_s e_0} + d_{c_s e_1} - d_{e_0 e_1}
$$
$$
e \in E
$$
$$
\forall E \subset T, |E| = 2
$$

$E$ represents an edge of the tour $T$, which has only the two cities that form that edge, being $e_0$ and $e_1$.

In [10]:
def cost(selected_city, edge):
    '''Calculates the cost of inserting the selected city in the edge.'''
    t0, t1 = edge
    distance_cs_t0 = distances[selected_city][t0]
    distance_cs_t1 = distances[selected_city][t1]
    distance_t0_t1 = distances[t0][t1]
    return distance_cs_t0 + distance_cs_t1 - distance_t0_t1

## Loop

Now we will enter in a loop to check which remaining cities, $\forall c \in \overline{C}$, to visit next and add them in an order to our tour $T$.

The body of the whole loop looks as follows:

while $\bar{C} \ne \emptyset$ do  
$\hbox{}\qquad c_s \leftarrow \newcommand{\argmin}{\mathop{\mathrm{argmin}}\limits} \argmin_{c \in \bar{C}} \newcommand{\dist}{\mathop{\mathrm{dist}}} \{\dist (c)\}$  
$\hbox{}\qquad k \leftarrow \newcommand{\argmin}{\mathop{\mathrm{argmin}}\limits} \argmin_{E \subset T} \newcommand{\cost}{\mathop{\mathrm{cost}}} \{\cost (c_s, E)\}$  
$\hbox{}\qquad T (t_1, \ldots, t_n) \leftarrow T (t_1, \ldots, t_k, c_s, t_{k+1}, \ldots, t_n)$  
$\hbox{}\qquad \bar{C} \leftarrow \bar{C} \setminus \{c_s\}$  
end while

To calculate $c_s \leftarrow \newcommand{\argmin}{\mathop{\mathrm{argmin}}\limits} \argmin_{c \in \bar{C}} \newcommand{\dist}{\mathop{\mathrm{dist}}} \{\dist (c)\}$, we will use the following function which will help us getting the actual nearest city:

In [11]:
def nearest():
    '''Returns a list of distances between each city and the subtour.'''
    return [(index, dist(city)) for index, city in enumerate(cities_bar)]

In the same way, to calculate $k \leftarrow \newcommand{\argmin}{\mathop{\mathrm{argmin}}\limits} \argmin_{E \subset T} \newcommand{\cost}{\mathop{\mathrm{cost}}} \{\cost (c_s, E)\}$, we will use the following function:

In [12]:
def argmin_cost(city):
    '''Returns a list of costs of each city in the subtour.'''
    indexed_costs = list()
    for i in range(len(tour) - 1):
        edge = (tour[i], tour[i + 1])
        indexed_costs.append((i, cost(city, edge)))
    return indexed_costs

And finally, to update the tour we will use a function to insert the selected city $c_s$, obtained with `nearest()`, next to the city $t_k$ in the subtour that has the smallest cost, obtained with `argmin_cost()`, as follows:

In [13]:
def update_tour(city, k):
    '''Inserts the selected city in the k index of the tour.'''
    index = k + 1
    global tour
    tour = tour[:index] + [city] + tour[index:]
    return tour

### First iteration

As we already know that we need to visit six more cities to complete the tour, we can start by checking which unvisited city is the closest from our subtour:

In [14]:
nearest()

[(0, 172), (1, 151), (2, 207), (3, 156), (4, 73), (5, 184)]

We can see that the city $4$ is the *nearest* to the subtour, $\therefore$ $c_s = 4$.

Now we want to know the index of the city in the subtour that will be previous to $4$. In this case, because we are in the first iteration, there's only one edge available, so we can insert $4$ between $6$ and $7$ or append it before the end; in both cases, the total length of the subtour would be the same because the cost is the same. Let's append it right after $7$:

$T \leftarrow T \oplus \{4\}$

In [15]:
# -2 means the index of penultimate city
update_tour(4, -2)

[6, 7, 4, 6]

$\bar{C}$ needs to be updated as the subtour $T$ grows:

$\bar{C} \leftarrow \bar{C} \setminus \{4\}$

In [16]:
cities_bar.remove(4)
cities_bar

[0, 1, 2, 3, 5]

### Second iteration

Let's choose the nearest city to our subtour $T$:

In [17]:
nearest()

[(0, 205), (1, 216), (2, 282), (3, 196), (4, 246)]

The city $3$ is the closest one, with a total distance of $196$. Now let's find the best edge to break from $T$:

In [18]:
argmin_cost(3)

[(0, 90), (1, 75), (2, 64)]

The edge between cities $t_2$ and $t_3$ is the cheapest one to break. These cities are $4$ and $6$, which means that the selected edge is the last one of the tour:

In [19]:
update_tour(3, 2)

[6, 7, 4, 3, 6]

Now we must update $\bar{C}$ by removing the city $3$:

$\bar{C} \leftarrow \bar{C} \setminus \{3\}$

In [20]:
cities_bar.remove(3)
cities_bar

[0, 1, 2, 5]